In [61]:
# To load the Pandas libraries as an alias 'pds' 
import pandas as pds
# To load the numpy libraries as an alias 'npy' 
import numpy as npy
# Importing Label Encoder for encoding categorical feature
from sklearn.preprocessing import LabelEncoder
import math
from decimal import Decimal
from sklearn.preprocessing import StandardScaler

#Function to Calculate Euclidean Distance
def Calc_Euclidean_Dist(TestData, TrainData):
    dist = 0
    for i in range(TrainData.shape[0]):
        dist = dist + (TestData[i] - TrainData[i])**2
    return math.sqrt(dist)

#Function to Calculate Manhattan Distance
def Calc_Manhattan_Dist(TestData, TrainData):
    dist = 0
    for i in range(TrainData.shape[0]):
        dist = dist + abs(TrainData[i] - TestData[i])
    return dist

# Function to calculate mean squared error used to calculate RMSE value
def mse(actual: npy.ndarray, predicted: npy.ndarray):
    return npy.mean(npy.square(actual - predicted))/len(actual)

# RMSE(Root Mean Squared Error) function to calculate accuracy of the model
def Calc_Accuracy(true_value, predicted_value):
    return npy.sqrt(mse(true_value, predicted_value)) 

# Function to get weights for applying weighted KNN
def  Weight(A):
    return 1/A**2
    if 0.001>A:
        return 1.0
    else:
        return 1/A**2
    
# Function to to apply Weighted KNN algorithm on continuous features
def weighted_knn_num(TrainData, TrainLabels, TestData, col_name, dist_methd):
    # Array initialisation to get the predicted labels and apply distance measures
    predicted_labels = []
    for i in range(len(TestData)): 
        dist_list = []
        for j in range(len(TrainData)):
            # If distance measure is Euclidean call function to calculate Euclidean distance
            if (dist_methd == 'Euclidean'):
                dist = Calc_Euclidean_Dist(TestData[i], TrainData[j])
            # If distance measure is Manhattan call function to calculate Manhattan distance
            elif (dist_methd == 'Manhattan'):
                dist = Calc_Manhattan_Dist(TestData[i], TrainData[j])
            # Assigning weights
            dist_list.append([Weight(dist),TrainLabels[j]])
        
        #numerical
        # Adding up the weights for each label and dividing by the individual weight to return respective predicted labels
        sigma_WL=0
        sigma_W=0
        for it in dist_list:
            w=it[0]
            l=it[1]
            sigma_WL+=w*l
            sigma_W+=w
        
        ans = sigma_WL/sigma_W
           
        predicted_labels.append(ans)
    return predicted_labels

# Function to to apply Weighted KNN algorithm on categorical features
def weighted_knn_cat(TrainData,TrainLabels, TestData, col_name, dist_methd):
    # Array initialisation to get the predicted labels and apply distance measures
    predicted_labels = []
    for i in range(len(TestData)): 
        dist_list = []
        for j in range(len(TrainData)):
            # If distance measure is Euclidean call function to calculate Euclidean distance
            if (dist_methd == 'Euclidean'):
                dist = Calc_Euclidean_Dist(TestData[i], TrainData[j])
            # If distance measure is Manhattan call function to calculate Manhattan distance
            elif (dist_methd == 'Manhattan'):
                dist = Calc_Manhattan_Dist(TestData[i], TrainData[j])
            # Assigning weights
            dist_list.append([Weight(dist),TrainLabels[j]])
  
        #catagorical
        # Adding up the weights for each label to count votes and return respective predicted labels
        votes = {}
        for it in dist_list:
            w=it[0]
            l=it[1]
            il = w*l
            if il in votes.keys():
                votes[l]+=w
            else: 
                votes[l]=w
                
        #print(votes)
        # Getting label and value of the member which has maximum voting count
        maxx_count = 0
        maxx_count_label = 0
        for label,count in votes.items():
            if count >= maxx_count:
                maxx_count_label = label
                maxx_count = count           
        
        ans = maxx_count_label
            
        predicted_labels.append(ans)
    return predicted_labels

# Function to to apply KNN algorithm on categorical features
def knn_cat(TrainData,TrainLabels, TestData, k, col_name, dist_methd):
    # Array initialisation to get the predicted labels and apply distance measures
    predicted_labels = []
    for i in range(len(TestData)): 
        dist_list = []
        for j in range(len(TrainData)):
            # If distance measure is Euclidean call function to calculate Euclidean distance
            if (dist_methd == 'Euclidean'):
                dist = Calc_Euclidean_Dist(TestData[i], TrainData[j])
            # If distance measure is Manhattan call function to calculate Manhattan distance
            elif (dist_methd == 'Manhattan'):            
                dist = Calc_Manhattan_Dist(TestData[i], TrainData[j])
            
            dist_list.append([dist,TrainLabels[j]])

        dist_list.sort()
        #predicted_labels.append(ans)   

        #Initializing a dictionary to store key value pairs of the list and apply concept of voting within list members
        votes = {}
        for x in range(K):
            if dist_list[x][1] in votes.keys():
                # incrementing the count of votes for that value if it exists
                votes[dist_list[x][1]] = votes[dist_list[x][1]] + 1
            else: 
                # initializing the count of votes for that value if it does not exist
                votes[dist_list[x][1]]=1
        
        # Getting label and value of the member which has maximum voting count
        maxx_count = 0
        maxx_count_label = 0
        for label,count in votes.items():
            if count >= maxx_count:
                maxx_count_label = label
                maxx_count = count
            
        predicted_labels.append(maxx_count_label)
    return predicted_labels

# Function to to apply KNN algorithm on continuous features
def knn_num(TrainData,TrainLabels, TestData, k, col_name, dist_methd):
    # Array initialisation to get the predicted labels and apply distance measures
    predicted_labels = []
    for i in range(len(TestData)): 
        dist_list = []
        for j in range(len(TrainData)):
            # If distance measure is Euclidean call function to calculate Euclidean distance
            if (dist_methd == 'Euclidean'):
                dist = Calc_Euclidean_Dist(TestData[i], TrainData[j])
            # If distance measure is Manhattan call function to calculate Manhattan distance
            elif (dist_methd == 'Manhattan'):
                dist = Calc_Manhattan_Dist(TestData[i], TrainData[j])
            dist_list.append([dist,TrainLabels[j]])


        dist_list.sort()
        # Adding up the weights for each label and dividing by the individual weight to return respective predicted labels        
        ans=0
        for i in range(K):
            ans += dist_list[i][1]
        ans = ans/K

        predicted_labels.append(ans) 
        
    return predicted_labels




In [62]:
for K in [1,7,'Weight']:
    for col_name in ['Species','Weight','Height','Width']:
        for dist_methd in ['Euclidean','Manhattan']:
            
            #loading data
            Fish = pds.read_csv("Fish.csv")
            
            # Using encoding technique for categorical feature 'Species' to calculate distance measures
            label_encoder = LabelEncoder()
            Fish["Species"] = label_encoder.fit_transform(Fish["Species"])

            #Creating missingness in data by removing 5% values from dataset for all features
            Fish[col_name + '_N'] = Fish[col_name].sample(frac=0.95)
            
            # Using Standard Scaler to Standardize the features by removing the mean and scaling them to unit variance
            scaler = StandardScaler()
            col_list = list(Fish.columns)
            col_list.remove(col_name)
            col_list.remove(col_name+'_N')
            Fish[col_list] = scaler.fit_transform(Fish[col_list])
            
            # Taking non null values from in train and null values in test
            train = Fish[pds.isnull(Fish[col_name + '_N']) == False]
            test = Fish[pds.isnull(Fish[col_name + '_N'])]

            # Dropping the feature for which we created missingness in data to as we need to calculate distance considering other features except that one
            train_features = train.drop([col_name,col_name + '_N'],axis=1).values
            train_labels =  train[col_name].values
            
            # Dropping the feature for which we created missingness in data to as we need to calculate distance considering other features except that one
            test_features = test.drop([col_name,col_name + '_N'],axis=1).values
            test_labels =  test[col_name].values
            
            if K == 'Weight':
                if (col_name == 'Species'):
                    #print('Predicted labels for ' , col_name ,' using Weighted KNN and ', dist_methd, ' Distance: ' , weighted_knn_cat(train_features,train_labels, test_features, col_name, dist_methd))
                    ypred = weighted_knn_cat(train_features,train_labels, test_features, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Weight'):
                    #print('Predicted labels for ' , col_name ,' using Weighted KNN and ' , dist_methd, 'Distance: ' , weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd))
                    ypred = weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Height'):
                    #print('Predicted labels for ' , col_name ,' using Weighted KNN and ' , dist_methd, 'Distance: ' , weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd))
                    ypred = weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Width'):
                    #print('Predicted labels for ' , col_name ,' using Weighted KNN and ' , dist_methd, 'Distance: ' , weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd))   
                    ypred = weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
            else:
                if (col_name == 'Species'):
                    #print('Predicted labels for ' , col_name ,' using ', K,'NN and ', dist_methd, ' Distance: ' , knn_cat(train_features,train_labels, test_features, K, col_name, dist_methd))
                    ypred = knn_cat(train_features,train_labels, test_features, K, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Weight'):
                    #print('Predicted labels for ' , col_name ,' using ', K,'NN and ', dist_methd, ' Distance: ' , knn_num(train_features,train_labels, test_features, K, col_name, dist_methd))
                    ypred = knn_num(train_features,train_labels, test_features, K, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Height'):
                    #print('Predicted labels for ' , col_name ,' using ', K,'NN and ', dist_methd, ' Distance: ' , knn_num(train_features,train_labels, test_features, K, col_name, dist_methd))
                    ypred = knn_num(train_features,train_labels, test_features, K, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Width'):
                    #print('Predicted labels for ' , col_name ,' using ', K,'NN and ', dist_methd, ' Distance: ' , knn_num(train_features,train_labels, test_features, K, col_name, dist_methd))
                    ypred = knn_num(train_features,train_labels, test_features, K, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)



Accuracy for feature Species using  Euclidean Distance:  0.4330127018922193
Accuracy for feature Species using  Manhattan Distance:  0.649519052838329
Accuracy for feature Weight using  Euclidean Distance:  19.451341534711688
Accuracy for feature Weight using  Manhattan Distance:  18.949967018440955
Accuracy for feature Height using  Euclidean Distance:  0.22487393829654845
Accuracy for feature Height using  Manhattan Distance:  0.19643317128542723
Accuracy for feature Width using  Euclidean Distance:  0.06584237332447855
Accuracy for feature Width using  Manhattan Distance:  0.10345081560819137
Accuracy for feature Species using  Euclidean Distance:  0.5153882032022076
Accuracy for feature Species using  Manhattan Distance:  0.375
Accuracy for feature Weight using  Euclidean Distance:  27.473817835424356
Accuracy for feature Weight using  Manhattan Distance:  27.943170280161894
Accuracy for feature Height using  Euclidean Distance:  0.20009669979729863
Accuracy for feature Height usin

In [63]:
for K in [1,7,'Weight']:
    for col_name in ['Species','Weight','Height','Width']:
        for dist_methd in ['Euclidean','Manhattan']:
            
            #loading data
            Fish = pds.read_csv("Fish.csv") 
            
            # Using encoding technique for categorical feature 'Species' to calculate distance measures
            label_encoder = LabelEncoder()
            Fish["Species"] = label_encoder.fit_transform(Fish["Species"])

            #Creating missingness in data by removing 10% values from dataset for all features
            Fish[col_name + '_N'] = Fish[col_name].sample(frac=0.90)
            
            # Using Standard Scaler to Standardize the features by removing the mean and scaling them to unit variance
            scaler = StandardScaler()
            col_list = list(Fish.columns)
            col_list.remove(col_name)
            col_list.remove(col_name+'_N')
            Fish[col_list] = scaler.fit_transform(Fish[col_list])
            
            # Taking non null values from in train and null values in test
            train = Fish[pds.isnull(Fish[col_name + '_N']) == False]
            test = Fish[pds.isnull(Fish[col_name + '_N'])]

            # Dropping the feature for which we created missingness in data to as we need to calculate distance considering other features except that one
            train_features = train.drop([col_name,col_name + '_N'],axis=1).values
            train_labels =  train[col_name].values

            # Dropping the feature for which we created missingness in data to as we need to calculate distance considering other features except that one
            test_features = test.drop([col_name,col_name + '_N'],axis=1).values
            test_labels =  test[col_name].values
            
            if K == 'Weight':
                if (col_name == 'Species'):
                    #print('Predicted labels for ' , col_name ,' using Weighted KNN and ', dist_methd, ' Distance: ' , weighted_knn_cat(train_features,train_labels, test_features, col_name, dist_methd))
                    ypred = weighted_knn_cat(train_features,train_labels, test_features, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Weight'):
                    #print('Predicted labels for ' , col_name ,' using Weighted KNN and ' , dist_methd, 'Distance: ' , weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd))
                    ypred = weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Height'):
                    #print('Predicted labels for ' , col_name ,' using Weighted KNN and ' , dist_methd, 'Distance: ' , weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd))
                    ypred = weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Width'):
                    #print('Predicted labels for ' , col_name ,' using Weighted KNN and ' , dist_methd, 'Distance: ' , weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd))  
                    ypred = weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
            else:
                if (col_name == 'Species'):
                    #print('Predicted labels for ' , col_name ,' using ', K,'NN and ', dist_methd, ' Distance: ' , knn_cat(train_features,train_labels, test_features, K, col_name, dist_methd))
                    ypred = knn_cat(train_features,train_labels, test_features, K, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Weight'):
                    #print('Predicted labels for ' , col_name ,' using ', K,'NN and ', dist_methd, ' Distance: ' , knn_num(train_features,train_labels, test_features, K, col_name, dist_methd))
                    ypred = knn_num(train_features,train_labels, test_features, K, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Height'):
                    #print('Predicted labels for ' , col_name ,' using ', K,'NN and ', dist_methd, ' Distance: ' , knn_num(train_features,train_labels, test_features, K, col_name, dist_methd))
                    ypred = knn_num(train_features,train_labels, test_features, K, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Width'):
                    #print('Predicted labels for ' , col_name ,' using ', K,'NN and ', dist_methd, ' Distance: ' , knn_num(train_features,train_labels, test_features, K, col_name, dist_methd))
                    ypred = knn_num(train_features,train_labels, test_features, K, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)



Accuracy for feature Species using  Euclidean Distance:  0.30618621784789724
Accuracy for feature Species using  Manhattan Distance:  0.34798527267687634
Accuracy for feature Weight using  Euclidean Distance:  25.50660836552951
Accuracy for feature Weight using  Manhattan Distance:  32.117754533123886
Accuracy for feature Height using  Euclidean Distance:  0.17858520812906098
Accuracy for feature Height using  Manhattan Distance:  0.13584180079691968
Accuracy for feature Width using  Euclidean Distance:  0.11722188228952823
Accuracy for feature Width using  Manhattan Distance:  0.08879980644686117
Accuracy for feature Species using  Euclidean Distance:  0.15309310892394862
Accuracy for feature Species using  Manhattan Distance:  0.19764235376052372
Accuracy for feature Weight using  Euclidean Distance:  18.380738140733474
Accuracy for feature Weight using  Manhattan Distance:  48.61781736337586
Accuracy for feature Height using  Euclidean Distance:  0.15799827659366691
Accuracy for fea

In [64]:
for K in [1,7,'Weight']:
    for col_name in ['Species','Weight','Height','Width']:
        for dist_methd in ['Euclidean','Manhattan']:
            
            #loading data
            Fish = pds.read_csv("Fish.csv") 
            
            # Using encoding technique for categorical feature 'Species' to calculate distance measures
            label_encoder = LabelEncoder()
            Fish["Species"] = label_encoder.fit_transform(Fish["Species"])

            #Creating missingness in data by removing 20% values from dataset for all features
            Fish[col_name + '_N'] = Fish[col_name].sample(frac=0.80)
            
            # Using Standard Scaler to Standardize the features by removing the mean and scaling them to unit variance
            scaler = StandardScaler()
            col_list = list(Fish.columns)
            col_list.remove(col_name)
            col_list.remove(col_name+'_N')
            Fish[col_list] = scaler.fit_transform(Fish[col_list])
            
            # Taking non null values from in train and null values in test
            train = Fish[pds.isnull(Fish[col_name + '_N']) == False]
            test = Fish[pds.isnull(Fish[col_name + '_N'])]

            # Dropping the feature for which we created missingness in data to as we need to calculate distance considering other features except that one
            train_features = train.drop([col_name,col_name + '_N'],axis=1).values
            train_labels =  train[col_name].values

            # Dropping the feature for which we created missingness in data to as we need to calculate distance considering other features except that one
            test_features = test.drop([col_name,col_name + '_N'],axis=1).values
            test_labels =  test[col_name].values
            
            if K == 'Weight':
                if (col_name == 'Species'):
                    #print('Predicted labels for ' , col_name ,' using Weighted KNN and ', dist_methd, ' Distance: ' , weighted_knn_cat(train_features,train_labels, test_features, col_name, dist_methd))
                    ypred = weighted_knn_cat(train_features,train_labels, test_features, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Weight'):
                    #print('Predicted labels for ' , col_name ,' using Weighted KNN and ' , dist_methd, 'Distance: ' , weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd))
                    ypred = weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Height'):
                    #print('Predicted labels for ' , col_name ,' using Weighted KNN and ' , dist_methd, 'Distance: ' , weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd))
                    ypred = weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Width'):
                    #print('Predicted labels for ' , col_name ,' using Weighted KNN and ' , dist_methd, 'Distance: ' , weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd))   
                    ypred = weighted_knn_num(train_features,train_labels, test_features, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
            else:
                if (col_name == 'Species'):
                    #print('Predicted labels for ' , col_name ,' using ', K,'NN and ', dist_methd, ' Distance: ' , knn_cat(train_features,train_labels, test_features, K, col_name, dist_methd))
                    ypred = knn_cat(train_features,train_labels, test_features, K, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Weight'):
                    #print('Predicted labels for ' , col_name ,' using ', K,'NN and ', dist_methd, ' Distance: ' , knn_num(train_features,train_labels, test_features, K, col_name, dist_methd))
                    ypred = knn_num(train_features,train_labels, test_features, K, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Height'):
                    #print('Predicted labels for ' , col_name ,' using ', K,'NN and ', dist_methd, ' Distance: ' , knn_num(train_features,train_labels, test_features, K, col_name, dist_methd))
                    ypred = knn_num(train_features,train_labels, test_features, K, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)
                elif (col_name == 'Width'):
                    #print('Predicted labels for ' , col_name ,' using ', K,'NN and ', dist_methd, ' Distance: ' , knn_num(train_features,train_labels, test_features, K, col_name, dist_methd))
                    ypred = knn_num(train_features,train_labels, test_features, K, col_name, dist_methd)
                    scc = Calc_Accuracy(test_labels, npy.array(ypred))
                    print('Accuracy for feature',col_name,'using ',dist_methd,'Distance: ',scc)



Accuracy for feature Species using  Euclidean Distance:  0.24206145913796356
Accuracy for feature Species using  Manhattan Distance:  0.13258252147247765
Accuracy for feature Weight using  Euclidean Distance:  22.00300716308455
Accuracy for feature Weight using  Manhattan Distance:  31.312631923374788
Accuracy for feature Height using  Euclidean Distance:  0.14468451569546498
Accuracy for feature Height using  Manhattan Distance:  0.09172309827852798
Accuracy for feature Width using  Euclidean Distance:  0.08411605165772093
Accuracy for feature Width using  Manhattan Distance:  0.05916306187668958
Accuracy for feature Species using  Euclidean Distance:  0.2460627460628691
Accuracy for feature Species using  Manhattan Distance:  0.1711632992203644
Accuracy for feature Weight using  Euclidean Distance:  19.26651635722059
Accuracy for feature Weight using  Manhattan Distance:  12.307642698658082
Accuracy for feature Height using  Euclidean Distance:  0.11220647679039446
Accuracy for featu

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in double_scalars
